In [70]:
import requests
from bs4 import BeautifulSoup
import datetime

def get_page(link):
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.1.6) Gecko/20091201 Firefox/3.5.6'} 
    r = requests.get(link, headers = headers)
    html = r.content
    html = html.decode('utf-8')
    soup = BeautifulSoup(html, 'lxml')
    return soup

def get_data(post_list):
    data_list =[]
    for post in post_list:
        title_td = post.find('td',class_='p_title')
        title = title_td.find('a', id=True).text.strip()
        post_link = title_td.find('a', id=True)['href']
        post_link = 'https://bbs.hupu.com' + post_link

        author = post.find('td',class_='p_author').a.text.strip()
        author_page = post.find('td',class_='p_author').a['href']
        start_date = post.find('td',class_='p_author').contents[2]
        start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()

        reply_view = post.find('td',class_='p_re').text.strip()
        reply = reply_view.split('/')[0].strip()
        view = reply_view.split('/')[1].strip()

        reply_time = post.find('td',class_='p_retime').a.text.strip()
        last_reply = post.find('td',class_='p_retime').contents[2]
        if ':' in reply_time:
            date_time = str(datetime.date.today()) + ' ' + reply_time
            date_time = datetime.datetime.strptime(date_time, '%Y-%m-%d %H:%M')
        else:
            date_time = datetime.datetime.strptime('2017-' + reply_time, '%Y-%m-%d').date()
        data_list.append([title, post_link, author, author_page, start_date, reply, view, last_reply, date_time])
    return data_list

link = "https://bbs.hupu.com/bxj"
soup = get_page(link)
post_list = soup.find_all('tr', mid=True)
data_list = get_data(post_list)
for each in data_list:
    print (each)

['【热线】2017全国高考已经开始，考生加油！考完回步行街报道吧！（庆学府路成立）', 'https://bbs.hupu.com/19338570.html', 'assholeeric', 'https://my.hupu.com/18199551720363', datetime.date(2017, 6, 7), '4184', '366724', 'heat520', datetime.datetime(2017, 6, 7, 22, 27)]
['考不上大学还可以去做什么？', 'https://bbs.hupu.com/19387841.html', '陈恒趼', 'https://my.hupu.com/20484540220867', datetime.date(2017, 6, 7), '5', '37', '說掰掰muea', datetime.datetime(2017, 6, 7, 22, 39)]
['粗大事啦！德国联邦卫生局官方宣布，打~飞机是无害的！', 'https://bbs.hupu.com/19387215.html', '莱莉库里我老婆', 'https://my.hupu.com/209781776426987', datetime.date(2017, 6, 7), '71', '12934', 'z793155', datetime.datetime(2017, 6, 7, 22, 39)]
['洗面奶而论，男jr喜欢哪种？', 'https://bbs.hupu.com/19385502.html', '广州第十人民医院', 'https://my.hupu.com/174021675065507', datetime.date(2017, 6, 7), '41', '2264', 'RongGor', datetime.datetime(2017, 6, 7, 22, 39)]
['点球挑战布冯的帖子想到的', 'https://bbs.hupu.com/19382922.html', '人人一口羊', 'https://my.hupu.com/53611729933466', datetime.date(2017, 6, 7), '146', '9151', '戅佬金', datetime.datetime(2

In [53]:
from pymongo import MongoClient
class MongoAPI(object):
    def __init__(self, db_ip, db_port, db_name, table_name):
        self.db_ip = db_ip
        self.db_port = db_port
        self.db_name = db_name
        self.table_name = table_name
        self.conn = MongoClient(host=self.db_ip, port=self.db_port)
        self.db = self.conn[self.db_name]
        self.table = self.db[self.table_name]
    def get_one(self, query):
        return self.table.find_one(query, projection={"_id": False})
    def get_all(self, query):
        return self.table.find(query)
    def add(self, kv_dict):
        return self.table.insert(kv_dict)
    def delete(self, query):
        return self.table.delete_many(query)
    def check_exist(self, query):
        ret = self.table.find_one(query)
        return ret != None
    # 如果没有会新建
    def update(self, query, kv_dict):
            self.table.update_one(query,{
              '$set': kv_dict
            }, upsert=True)

In [71]:
hupu_post = MongoAPI("localhost",  27017,  "hupu", "post")
for each in data_list:
    hupu_post.add({"title": each[0], 
                "post_link": each[1],
               "author": each[2],
               "author_page": each[3],
               "start_date": str(each[4]),
               "reply": each[5],
               "view": each[6],
               "last_reply": each[7],
               "last_reply_time": str(each[8])})

['【热线】2017全国高考已经开始，考生加油！考完回步行街报道吧！（庆学府路成立）', 'https://bbs.hupu.com/19338570.html', 'assholeeric', 'https://my.hupu.com/18199551720363', datetime.date(2017, 6, 7), '4184', '366724', 'heat520', datetime.datetime(2017, 6, 7, 22, 27)]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:17: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


['考不上大学还可以去做什么？', 'https://bbs.hupu.com/19387841.html', '陈恒趼', 'https://my.hupu.com/20484540220867', datetime.date(2017, 6, 7), '5', '37', '說掰掰muea', datetime.datetime(2017, 6, 7, 22, 39)]
['粗大事啦！德国联邦卫生局官方宣布，打~飞机是无害的！', 'https://bbs.hupu.com/19387215.html', '莱莉库里我老婆', 'https://my.hupu.com/209781776426987', datetime.date(2017, 6, 7), '71', '12934', 'z793155', datetime.datetime(2017, 6, 7, 22, 39)]
['洗面奶而论，男jr喜欢哪种？', 'https://bbs.hupu.com/19385502.html', '广州第十人民医院', 'https://my.hupu.com/174021675065507', datetime.date(2017, 6, 7), '41', '2264', 'RongGor', datetime.datetime(2017, 6, 7, 22, 39)]
['点球挑战布冯的帖子想到的', 'https://bbs.hupu.com/19382922.html', '人人一口羊', 'https://my.hupu.com/53611729933466', datetime.date(2017, 6, 7), '146', '9151', '戅佬金', datetime.datetime(2017, 6, 7, 22, 39)]
['【我是你心里的一首歌——音乐区专属聊天大楼】乐你想乐', 'https://bbs.hupu.com/3966486.html', 'dandanaw', 'https://my.hupu.com/dandanaw', datetime.date(2012, 7, 10), '48117', '199269', 'RaulGonzaIez', datetime.datetime(2017, 6, 7, 22, 39

In [72]:
import requests
from bs4 import BeautifulSoup
import datetime
from pymongo import MongoClient
import time

hupu_post = MongoAPI("localhost",  27017,  "hupu", "post")
for i in range(1,101):
    link = "https://bbs.hupu.com/bxj-" + str(i)
    soup = get_page(link)
    
    post_list = soup.find_all('tr', mid=True)
    data_list = get_data(post_list)
    for each in data_list:
        hupu_post.update({"post_link": each[1]},{"title": each[0], 
                    "post_link": each[1],
                   "author": each[2],
                   "author_page": each[3],
                   "start_date": str(each[4]),
                   "reply": each[5],
                   "view": each[6],
                   "last_reply": each[7],
                   "last_reply_time": str(each[8])})

    time.sleep(3)
    print ('第', i ,'页获取完成，休息3秒')

第 1 页获取完成，休息3秒
第 2 页获取完成，休息3秒
第 3 页获取完成，休息3秒
第 4 页获取完成，休息3秒
第 5 页获取完成，休息3秒
第 6 页获取完成，休息3秒
第 7 页获取完成，休息3秒
第 8 页获取完成，休息3秒
第 9 页获取完成，休息3秒
第 10 页获取完成，休息3秒
第 11 页获取完成，休息3秒
第 12 页获取完成，休息3秒
第 13 页获取完成，休息3秒
第 14 页获取完成，休息3秒
第 15 页获取完成，休息3秒
第 16 页获取完成，休息3秒
第 17 页获取完成，休息3秒
第 18 页获取完成，休息3秒
第 19 页获取完成，休息3秒
第 20 页获取完成，休息3秒
第 21 页获取完成，休息3秒
第 22 页获取完成，休息3秒
第 23 页获取完成，休息3秒
第 24 页获取完成，休息3秒
第 25 页获取完成，休息3秒
第 26 页获取完成，休息3秒
第 27 页获取完成，休息3秒
第 28 页获取完成，休息3秒
第 29 页获取完成，休息3秒
第 30 页获取完成，休息3秒
第 31 页获取完成，休息3秒
第 32 页获取完成，休息3秒
第 33 页获取完成，休息3秒
第 34 页获取完成，休息3秒
第 35 页获取完成，休息3秒
第 36 页获取完成，休息3秒
第 37 页获取完成，休息3秒
第 38 页获取完成，休息3秒
第 39 页获取完成，休息3秒
第 40 页获取完成，休息3秒
第 41 页获取完成，休息3秒
第 42 页获取完成，休息3秒
第 43 页获取完成，休息3秒
第 44 页获取完成，休息3秒
第 45 页获取完成，休息3秒
第 46 页获取完成，休息3秒
第 47 页获取完成，休息3秒
第 48 页获取完成，休息3秒
第 49 页获取完成，休息3秒
第 50 页获取完成，休息3秒
第 51 页获取完成，休息3秒
第 52 页获取完成，休息3秒
第 53 页获取完成，休息3秒
第 54 页获取完成，休息3秒
第 55 页获取完成，休息3秒
第 56 页获取完成，休息3秒
第 57 页获取完成，休息3秒
第 58 页获取完成，休息3秒
第 59 页获取完成，休息3秒
第 60 页获取完成，休息3秒
第 61 页获取完成，休息3秒
第 62 页获取完成，休息3秒
第 63 页获取完成，休息3秒
第